# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

%matplotlib inline


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = "../output_data/cities.csv"

cities_df = pd.read_csv(filepath)
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vammala,61.33,22.90,36.00,93,90,10.29,FI,1607308852
1,Rikitea,-23.12,-134.97,76.89,79,0,14.41,PF,1607308853
2,Riyadh,24.69,46.72,51.80,66,0,9.17,SA,1607308853
3,Cape Town,-33.93,18.42,64.99,87,0,12.75,ZA,1607308782
4,Carauari,-4.88,-66.90,73.58,90,99,1.95,BR,1607308853


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
gmaps.configure(api_key=g_key)

location = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"]

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(location, weights = humidity)
fig.add_layer(heatmap_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [16]:
ideal_df = cities_df[(cities_df["Max Temp"]< 80) & (cities_df["Max Temp"]> 70) & (cities_df["Wind Speed"]<10) & (cities_df["Cloudiness"]==0)]
ideal_clean = ideal_df.dropna()
print(len(ideal_clean))
ideal_clean.head()

15


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,Busselton,-33.65,115.33,79.00,30,0,4.50,AU,1607308854
23,Cayenne,4.93,-52.33,75.20,100,0,8.03,GF,1607308638
29,Hong Kong,22.29,114.16,71.01,60,0,1.01,HK,1607308648
102,Tura,25.52,90.22,70.52,66,0,0.98,IN,1607308689
163,Bodden Town,19.28,-81.25,77.00,94,0,5.50,KY,1607308869


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [88]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
types = "lodging"
keyword = "Hotel"
radius = 5000
hotel_list = []

for i in range(len(ideal_clean)):
    response = requests.get(f"{base_url}location={ideal_clean.iloc[i,1]},{ideal_clean.iloc[i,2]}&radius={radius}&type={types}&keyword={keyword}&key={g_key}").json()
    try:
        hotel_list.append(response["results"][0]["name"])
    except IndexError:
        hotel_list.append("No hotel within 5000m")



In [92]:
ideal_clean["Hotel Name"] = hotel_list
hotel_df = ideal_clean
hotel_df.head(15)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,Busselton,-33.65,115.33,79.00,30,0,4.50,AU,1607308854,Abbey Beach Resort
23,Cayenne,4.93,-52.33,75.20,100,0,8.03,GF,1607308638,Royal Amazonia
29,Hong Kong,22.29,114.16,71.01,60,0,1.01,HK,1607308648,The Harbourview
102,Tura,25.52,90.22,70.52,66,0,0.98,IN,1607308689,Hotel Polo Orchid
163,Bodden Town,19.28,-81.25,77.00,94,0,5.50,KY,1607308869,Turtle Nest Inn & Condos
240,Mogadishu,2.04,45.34,78.82,76,0,9.82,SO,1607308876,Dolphin Hotel
281,Kayes,14.00,-11.00,73.17,34,0,5.37,ML,1607308880,No hotel within 5000m
289,Takoradze,4.88,-1.76,78.60,83,0,6.67,GH,1607308881,Milgwen Hotel
303,Sur,22.57,59.53,74.95,59,0,7.70,OM,1607308882,Sur Plaza Hotel
436,Sittwe,20.15,92.90,72.03,56,0,4.74,MM,1607308894,Yuzana Aung Hotel


In [93]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [98]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))